# Here, we will be performing a grid search to look for the best hyperparameters for our model.


### We will set up our project much like in the previous notebook, but we will will cycle through different regression models and hyperparameters to find the best model for our data.

In [1]:
import numpy as np
import pandas as pd
import sys
import os 
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
import glob

from sglm import utils, glm_fit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet, Ridge, LinearRegression
import torch
import torch_linear_regression as tlr
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2

### Check GPU availability:

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")  ## For Macs with Mn chips
else:
    DEVICE = torch.device("cpu")
print(f"Device: {DEVICE}")

Device: cpu


## Create a project

#### First, let's create a new project. The project directory will create a data and results folder and a config file.

#### You will need to edit the config file with the particular glm params you wish to use. Fields that are necessary to edit are: predictors, predictors_shift_bounds, response, and the glm_keyword_args.

#### You will also need to move your data into the data folder.

In [3]:
project_name = 'test_glm'
project_dir = r'C:\Users\janet\Documents\orphan_code'

utils.create_new_project(project_name, project_dir)

Project directory already exists!


'C:\\Users\\janet\\Documents\\orphan_code\\test_glm\\config.yaml'

In [4]:
project_path = os.path.join(project_dir, project_name)
files = os.listdir(project_path)

assert 'data' in files, 'data folder not found! {}'.format(files)
assert 'results' in files, 'results folder not found! {}'.format(files)
assert 'config.yaml' in files, 'config.yaml not found! {}'.format(files)

In [5]:
config_file = os.path.join(project_path, 'config.yaml')
config = utils.load_config(config_file)

# Import and Format Data

Input data should conform to the following convention and be saved as a *.csv:

Indices / Unique Row Identifiers:
* SessionName -- Any order is acceptable
* TrialNumber-- Must be in chronological order, but does not need to start from zero
* Timestamp -- Must be in chronological order, but does not need to start from zero

Columns (Predictors + Responses):
* Predictors - binary
* Reponses - e.g. neural responses (analog or binary)

Example, shown below is dummy data depicting a trial_0 that last four response timestamps:
| SessionName | TrialNumber | Timestamp | predictor_1 | predictor_2 | predictor_3 | response_1 | response_2 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| session_0 | trial_0 | -1 | 0 | 0 | 0 | 1 | 0.3 |
| session_0 | trial_0 | 0 | 0 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_0 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| session_0 | trial_0 | 2 | 0 | 1 | 0 | 1 | 0.3 |
| session_0 | trial_1 | -2 | 0 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_1 | -1 | 0 | 0 | 0 | 1 | 2.3 |
| session_0 | trial_1 | 0 | 1 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_1 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_0 | 5 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_0 | 6 | 1 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_0 | 7 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_0 | 8 | 0 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_1 | 9 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_1 | 10 | 0 | 0 | 0 | 1 | 2.3 |
....

#### If needed, use the following function to combine multiple sessions into one csv. You will need a filename you wish to call your output_csv

In [6]:
output_csv = 'combined.csv'

utils.combine_csvs(project_path, output_csv)

Output file already exists! Please remove or rename the existing file: C:\Users\janet\Documents\orphan_code\test_glm\data\combined.csv, defaulting to previous version


'C:\\Users\\janet\\Documents\\orphan_code\\test_glm\\data\\combined.csv'

#### Next, we'll load the data and set the columns you wish to use as fixed indices. Following this step, you can explore and add features/predictors to the dataframe as needed.

In [7]:
input_file = os.path.join(project_path, 'data', output_csv)
index_col = ['SessionName', 'TrialNumber', 'Timestamp']

df = utils.read_data(input_file, index_col)

print('Your dataframe has {} rows and {} columns'.format(df.shape[0], df.shape[1]))

Your dataframe has 262972 rows and 6 columns


#### Shift responses and predictors. If you do not want to shift your predictors by an amount you set, feel free to comment out the entire "predictors_shift_bounds" in config.yaml. We will then use the default set when we created the config file. 

#### For, larger datasets, you may want to sparse your training data. You can do this by seeting the sparsify argument to True in the shift_predcitors function.

In [9]:
response_shift, df_predictors_shift, shifted_params = glm_fit.shift_predictors(config, df, sparsify=False)
print('Your dataframe was shifted using: {}'.format(shifted_params))

Original length: 262972, Mask length: 262672
Your dataframe was shifted using: [('go', [-50, 100]), ('nogo', [-50, 100]), ('reward', [-50, 100]), ('lick', [-50, 100])]


In [10]:
X_train,X_test, y_train, y_test = glm_fit.split_data(df_predictors_shift, response_shift, config)

print('Training data has {} rows and {} columns'.format(X_train.shape[0], X_train.shape[1]))
print('Testing data has {} rows and {} columns'.format(X_test.shape[0], X_test.shape[1]))

Training data has 210137 rows and 604 columns
Testing data has 52535 rows and 604 columns


## Now, we will perform a grid search to find the best hyperparameters for our model. We will cycle through different regression models and hyperparameters to find the best model for our data. We will only loop through `Ridge()` and `OLS()` models given that these are implemented for GPU acceleration. *Note: PyTorch is also CPU compatible, so this notebook can be run on a CPU as well*.

In [12]:
#First, change to torch tensor
X_train_tensor = utils.df_to_tensor(X_train)
y_train_tensor = utils.df_to_tensor(y_train)
X_test_tensor = utils.df_to_tensor(X_test)
y_test_tensor = utils.df_to_tensor(y_test)

### Check the rank
The regression models use the closed form solutions for max performance, and if the input matrix is not full rank, the algorithm may fail. So, it is important to confirm that the input matrix is full rank before applying the regression models.

In [20]:
#Check the rank
print(f"Training tensor is full rank: {tlr.check_full_rank(X_train_tensor).item()}")

Training tensor is full rank: True


In [ ]:
#Begin with OLS - no tuning required
ols = tlr.OLS()
ols.fit(X_train_tensor, y_train_tensor)
ols_train_preds = ols.predict(X_train_tensor)
ols_test_preds = ols.predict(X_test_tensor)

ols_train_score = ols.score(X_test_tensor, y_test_tensor)
print('OLS Train Score: {}'.format(ols_train_score))

In [ ]:
# Ridge Regression - tune alpha using backpropogation

scores_sweep = {}
for alpha in np.geomspace(1e-5, 1e10, num=300): #loops through 300 alphas
    model_ridge_sweep = tlr.Ridge(alpha=alpha)
    model_ridge_sweep.fit(X_train_tensor, y_train_tensor)
    score = model_ridge_sweep.score(X_test_tensor, y_test_tensor)
    scores_sweep[alpha] = score

## Use backpropagation to fit the alpha parameter
fn_loss = torch.nn.MSELoss()
alpha_optimized = torch.nn.Parameter(torch.tensor(1.0))
optimizer = torch.optim.AdamW(params=[alpha_optimized], lr=0.3)
losses = []
for i in range(50):
    model_ridge_optimized = tlr.Ridge(alpha=alpha_optimized)
    model_ridge_optimized.fit(X_train_tensor, y_train_tensor)
    Y_pred = model_ridge_optimized.predict(X_test_tensor)
    loss = fn_loss(Y_pred, y_test_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
score_optimized = model_ridge_optimized.score(X_test_tensor, y_test_tensor).item()
print(f"Optimized Ridge Score: {score_optimized}, with alpha: {alpha_optimized.item()}")

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel("Iteration")
plt.ylabel("Loss")

plt.figure()
plt.plot(list(scores_sweep.keys()), list(scores_sweep.values()), marker='o')
plt.plot(alpha_optimized.item(), score_optimized, marker='o')
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.xscale('log')

In [ ]:
# Get best model and parameters between Ridge and OLS
if score_optimized > ols_train_score:
    best_model = model_ridge_optimized
    best_alpha = alpha_optimized.item()
    best_score = score_optimized
else:
    best_model = ols
    best_alpha = None
    best_score = ols_train_score

print(f"Best Model: {best_model}, Best Alpha: {best_alpha}, Best Score: {best_score}")

In [ ]:
# Save best model and parameters
grid_sweep = {'model': best_model, 'best_score': best_score, 'alpha_optimized': alpha_optimized, 'losses': losses}

import pickle
model_path = config['Project']['project_path'] + '/models'
model_name = 'grid_search' + '_model.pkl'
model_full_path = os.path.join(model_path, model_name)
with open(model_full_path, 'wb') as f:
    pickle.dump(grid_sweep, f)

In [ ]:
#for opening a saved model
model_dict_path = r'C:\Users\janet\Dropbox (HMS)\processed_photometry\9010\9010_base\9010_base\models\grid_search_model.pkl'
import pickle
with open(model_dict_path, 'rb') as f:
    grid_reg = pickle.load(f)

In [ ]:
# dump the best paramaters to config file
regression_type = best_model
alpha = alpha_optimized.item()

config['glm_params']['regression_type'] = regression_type
if regression_type == 'Ridge':
    config['glm_params']['glm_keyword_args']['ridge']['alpha'] = alpha

# save back to config file
cfg_file = os.path.join(project_dir, project_path, "config.yaml")
utils.save_to_yaml(config, cfg_file)


## Now we can run the recommended regression with the best parameters.

In [ ]:
model, y_pred, score, beta, intercept = glm_fit.fit_glm(config, X_train, X_test, y_train, y_test, pytorch=True)

## Save your outputs

In [ ]:
#Create your model dictonary, this should include all the information you wish to save
model_dict = {'model': model,
                'model_type': config['glm_params']['regression_type'],
                'y_pred': y_pred,
                'score': score,
                'beta': beta,
                'intercept': intercept,
                'X_train': X_train,
                'X_test': X_test,
                'y_train': y_train,
                'y_test': y_test,
              }

#Save your model dictionary
glm_fit.save_model(model_dict, config)

## Generate and save figures and results.
The following requires non-sparse data. If you have sparse data, you will need to re-run `shift_predictors` with the `sparse` argument set to `False`.

`plot_and_save` will save scatter plots of the predicted vs actual responses and the residuals and your beta coefficients. 

`plot_betas` will only *plot* the beta coefficients. 

`plot_aligned_dataStream` will plot the aligned data stream (e.g. aligned input data). You will need to run the `align_dataStream` function before running this plot.

`plot_actual_v_reconstructed` will plot the actual vs reconstructed responses. You will need to run the `align_reconstructed_dataStream` function before running this plot.

In [ ]:
response_shift, df_predictors_shift, shifted_params = glm_fit.shift_predictors(config, df, sparsify=False)
print('Your dataframe was shifted using: {}'.format(shifted_params))

In [ ]:
save_path = os.path.join(project_path, 'results')

In [ ]:
glm_fit.plot_and_save(config, y_pred, y_test, beta, df_predictors_shift)

In [ ]:
utils.plot_betas(config, beta, df_predictors_shift, shifted_params, save=None, save_path=None, show_plot=True)

### Align the data and plot the actual and reconstructed responses (e.g. predicted y) against the true responses (e.g. neural responses) for each prediction. 

In [ ]:
# Align your actual data
aligned_dataStream = utils.align_dataStream(config, df, shifted_params)

# Plot aligned data
utils.plot_aligned_dataStream(aligned_dataStream, config, save=True, save_path=save_path, reconstructed=False, show_plot=True)

In [ ]:
# Reconstruct your signal from your X-inputs
recon_dataStream = utils.align_reconstructed_dataStream(config, 
                                                        df, df_predictors_shift,
                                                         shifted_params, model)

# Plot reconstructed data
utils.plot_aligned_dataStream(recon_dataStream, config, save=True, save_path=save_path, reconstructed=True, show_plot=True)

In [ ]:
# Plot actual vs reconstructed
utils.plot_actual_v_reconstructed(config, aligned_dataStream, recon_dataStream, save=True, save_path=save_path, show_plot=True)

## For additional validation such as Leave-One-Out Cross-Validation and assessing train/test performance, please refer to the fitGLM notebook. 